# Homework 4

## Coding

Goal: Find complaint types that increased or decreased when COVID-19 hit New York Ciy: mid-March 2020.

### Step 0: Setup

For this homework, instead of the data being provided, you will export it directly from the NYC Open Data Portal, as if you were working on your own project.

1. Download the data.
   1. Visit the [311 data](https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9/data) page.
   1. From that page, filter the data to `Created Date`s between `01/01/2020 12:00:00 AM` and `03/31/2020 11:59:59 PM`.
   1. It should say "Showing 311 Service Requests 1-100 out of 548,184" near the bottom of the screen.
      - It's ok if the total is slightly different.
   1. Click `Export`.
   1. Click `CSV`. It will start downloading a file.
   1. Rename the file `311_covid.csv`.
1. Upload the CSV.
1. Read the data from `./<filename>.csv`.
   - You may need to adjust the path, depending on where the CSV/notebook are.

If the above is taking a long time due to have a slow network connection or whatever else, load the data from:

https://storage.googleapis.com/python-public-policy/data/311_covid.csv.zip

### Step 1: Load data

Read the data into a DataFrame called `df_2020`.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "notebook_connected+pdf"

In [2]:
# your code here
df_2020 = pd.read_csv('311_covid.csv')
df_2020.head()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
0,45288120,01/01/2020 12:00:00 AM,01/02/2020 09:51:29 AM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,10455.0,748 EAST 149 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.812996,-73.907973,"(40.81299645614164, -73.90797324533352)"
1,45285651,01/01/2020 12:00:00 AM,01/02/2020 12:00:01 AM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Restaurant/Bar/Deli/Bakery,10458.0,2701 DECATUR AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.864866,-73.888783,"(40.86486556770799, -73.88878325729915)"
2,45285347,01/01/2020 12:00:00 AM,01/10/2020 12:00:01 AM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Restaurant/Bar/Deli/Bakery,11229.0,3442 NOSTRAND AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.600129,-73.941843,"(40.6001292057807, -73.94184291675883)"
3,45285821,01/01/2020 12:00:00 AM,01/02/2020 09:50:09 AM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,1 or 2,Other (Explain Below),11203.0,5707 CHURCH AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.652536,-73.923540,"(40.65253575905768, -73.92353994017134)"
4,45289558,01/01/2020 12:00:00 AM,01/15/2020 12:00:01 AM,DOHMH,Department of Health and Mental Hygiene,Food Poisoning,3 or More,Other (Explain Below),11215.0,625 UNION STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.677963,-73.984436,"(40.677963041857886, -73.98443609121443)"


### Step 2: Convert dates

Copy code from [Lecture 4](lecture_4.ipynb) to convert the `Created Date` to a `datetime`.

In [5]:
# your code here
df_2020["Created Date"] = pd.to_datetime(df_2020["Created Date"], format="%m/%d/%Y %I:%M:%S %p")
df_2020.dtypes

Unique Key                                 int64
Created Date                      datetime64[ns]
Closed Date                               object
Agency                                    object
Agency Name                               object
Complaint Type                            object
Descriptor                                object
Location Type                             object
Incident Zip                             float64
Incident Address                          object
Street Name                               object
Cross Street 1                            object
Cross Street 2                            object
Intersection Street 1                     object
Intersection Street 2                     object
Address Type                              object
City                                      object
Landmark                                  object
Facility Type                             object
Status                                    object
Due Date            

### Step 3: Date counts

Create a DataFrame called `date_counts` that has the count of complaints per Complaint Type per day, then display it.

In [6]:
# your code here
date_counts= df_2020.groupby(['Complaint Type']).resample('D', on='Created Date').size().reset_index(name='count_requests')
date_counts

,Complaint Type,Created Date,count_requests
0,APPLIANCE,2020-01-01,6
1,APPLIANCE,2020-01-02,27
2,APPLIANCE,2020-01-03,29
3,APPLIANCE,2020-01-04,11
4,APPLIANCE,2020-01-05,12
...,...,...,...
13695,Window Guard,2020-03-22,0
13696,Window Guard,2020-03-23,0
13697,Window Guard,2020-03-24,0
13698,Window Guard,2020-03-25,1


### Step 4: Plotting over time

Create a line chart of the count of complaints over time, one line per `Complaint Type`.

In [7]:
# your code here
fig = px.line(date_counts,x="Created Date",y="count_requests", color = 'Complaint Type', title="Complaints Over Time")
fig.show()

______________________________________________________________________

This has the information we need, but is a lot to look at. Let's only show complaint types that changed greatly (in March 2020) relative to the same period in the previous year (March 2019).

### Step 5: March 2020 counts

Create a DataFrame called `mar_counts` that has the count of each `Complaint Type` in March 2020 in a column called `2020`. Use [`.to_frame()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.to_frame.html) (instead of [`.reset_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.reset_index.html)) to use the `Complaint Type` as the index. It should end up looking something like this:

Complaint Type | 2020
--- | ---
APPLIANCE | 824
Abandoned Vehicle | 2500
Air Quality | 657
… | …

_Note there is no numeric index._

In [20]:
# your code here
after_feb = df_2020['Created Date'] > "2020-02-29"
before_apr = df_2020['Created Date'] < "2020-04-01"



### Step 6: Get March 2019 data

Follow Steps 0-2 again, this time with 311 requests for all of March 2019. Name the DataFrame `mar_2019`.

Similar to Step 0, if having trouble downloading, you can load from:

https://storage.googleapis.com/python-public-policy/data/311_mar_2019.csv.zip

In [8]:
# your code here
mar_2019 = pd.read_csv('311_mar_2019.csv')
mar_2019["Created Date"] = pd.to_datetime(mar_2019["Created Date"], format="%m/%d/%Y %I:%M:%S %p")
mar_2019.dtypes

/tmp/ipykernel_47/716095295.py:2: DtypeWarning:

Columns (8,31) have mixed types. Specify dtype option on import or set low_memory=False.



Unique Key                                 int64
Created Date                      datetime64[ns]
Closed Date                               object
Agency                                    object
Agency Name                               object
Complaint Type                            object
Descriptor                                object
Location Type                             object
Incident Zip                              object
Incident Address                          object
Street Name                               object
Cross Street 1                            object
Cross Street 2                            object
Intersection Street 1                     object
Intersection Street 2                     object
Address Type                              object
City                                      object
Landmark                                  object
Facility Type                             object
Status                                    object
Due Date            

### Step 7: March 2019 counts

1. Get the `Complaint Type` counts for March 2019.
1. Add these to the `mar_counts` DataFrame as a column called `2019`.
   - Reminder that adding a Series as a new column to a DataFrame matches rows based on the index.

In [31]:
# your code here
complaints_mar_2019= mar_2019.groupby(['Complaint Type']).resample('D', on='Created Date').size().reset_index(name='count_requests')
complaints_mar_2019

,Complaint Type,Created Date,count_requests
0,APPLIANCE,2019-03-01,54
1,APPLIANCE,2019-03-02,14
2,APPLIANCE,2019-03-03,15
3,APPLIANCE,2019-03-04,47
4,APPLIANCE,2019-03-05,44
...,...,...,...
5413,Water System,2019-03-29,149
5414,Water System,2019-03-30,103
5415,Water System,2019-03-31,68
5416,Window Guard,2019-03-19,1


### Step 8: Percent change

Use `mar_counts` to calculate the percent change from March 2019 to March 2020 for each `Complaint Type`. Save as the `pct_change` column. Should result in something like this:

Complaint Type | 2020 | 2019 | pct_change
--- | --- | --- | ---
APPLIANCE | 824 | 1042 | -0.20
Abandoned Vehicle | 2500 | 1 | 2499.00
Air Quality | 657 | 642 | 0.02
… | … | … | …

In [8]:
# your code here

### Step 9: Filter

Filter to `Complaint Type`s that both:

- Occurred at least 50 times in March 2020
- Changed (increased _or_ decreased) by more than 90%

and save the DataFrame as `top_changed`. A couple of things that may be helpful:

- [Selecting Subsets of Data in Pandas](https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-39e811c81a0c#0eb4), starting from "Multiple condition expression"
- [Getting absolute values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.abs.html)

In [9]:
# your code here

### Step 10: Top changed

Filter the `date_counts` to only the `top_changed` `Complaint Type`s. Save as `top_changed_by_day`.

In [10]:
# your code here

### Step 11: Plotting changed complaints

Make a similar plot to Step 4, but with only the top complaints (`top_changed_by_day`).

In [11]:
# your code here

## Question 0

***Did the change of any of the `Complaint Type`s in Step 10/11 surprise you? Why or why not? (Speak at least one specifically.)***

YOUR RESPONSE HERE

Then, give these a read:

- [NY Daily News article](https://www.nydailynews.com/coronavirus/ny-coronavirus-price-gouging-new-york-city-20200429-z5zs4ygfxbcmrpgzfrnlbxsnea-story.html)
- [Press release from Department of Consumer and Worker Protection](https://www1.nyc.gov/site/dca/media/pr031720-DCWP-Emergency-Rule-Price-Gouging-Illegal.page)

Overall caveat for this assignment: [**correlation does not imply causation**](https://www.khanacademy.org/math/probability/scatterplots-a1/creating-interpreting-scatterplots/v/correlation-and-causality).

## Bonus: Charting against COVID-19 case counts

_0.4 points_

Let's take a look at the `Consumer Complaint`s against the COVID-19 case numbers in NYC in the same graph. You'll need to:

1. Find data that provides the COVID-19 case counts for NYC by day.
1. Create a DataFrame with only the `Consumer Complaint` `Complaint Type` counts, by day.
1. Chart the two against each other for February through March.

The result should look something like this (without the black box):

![bonus solution chart](extras/img/hw_4_bonus.png)

Some resources that may be helpful:

- [Reading CSV data from GitHub](https://projectosyo.wixsite.com/datadoubleconfirm/single-post/2019/04/15/Reading-csv-data-from-Github---Python)
- [Two Y Axes in plotly](https://plotly.com/python/multiple-axes/#two-y-axes)
   - Note that the `plotly.graph_objects` syntax is a bit different than the `plotly.express` syntax we've been using. With `go.Scatter()`, you don't provide the DataFrame and the names of the columns; you pass `x` and `y` as lists/Series of the values themselves.
- [Setting the Range of Axes Manually in plotly](https://plotly.com/python/axes/#setting-the-range-of-axes-manually)

In [12]:
# your code here

## Bonus Question 1: What observations do you have?

YOUR RESPONSE HERE

Now [turn in the assignment](https://python-public-policy.afeld.me/en/latest/README.html#turning-in-assignments).

## Tutorials

In the videos below, don't get hung up on mentions of JavaScript, Node.js, or Twilio — those were technologies used for another course.

1. Watch:
   1. [What are APIs?](https://www.youtube.com/watch?v=OVvTv9Hy91Q)
   1. [APIs, Conceptually](https://drive.google.com/file/d/10VCtYI5Im9MnvDcn4vnUeWbqztF77tyL/view?usp=sharing)
1. Read [Understanding And Using REST APIs](https://www.smashingmagazine.com/2018/01/understanding-using-rest-api/)
1. Watch:
   1. [Let's look at some data](https://drive.google.com/file/d/10_2UPxa0ThWus47jKKeefGji5ZZmnr-e/view?usp=sharing)
   1. [Data formats](https://drive.google.com/file/d/10dR1oMt7V-Hk75mkIpnguq70mukSz6OA/view?usp=sharing)
   1. [API documentation](https://drive.google.com/file/d/10fOxW42-ODIgHlLgLCP_wklrxB8G4K3A/view?usp=sharing)
1. Read [Python’s Requests Library (Guide)](https://realpython.com/python-requests/) through `The Message Body`

## Participation

Reminder about the [between-class participation requirement](https://python-public-policy.afeld.me/en/latest/syllabus.html#participation).